In [1]:
!pip install mlxtend


In [2]:
!pip install mlxtend --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.22.0
    Uninstalling mlxtend-0.22.0:
      Successfully uninstalled mlxtend-0.22.0


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import csv

In [4]:
train_csv=pd.read_csv('/content/drive/MyDrive/train (2).csv')
test_csv=pd.read_csv('/content/drive/MyDrive/test (2).csv')
train_csv=train_csv.drop(['Name','Cabin','Destination','HomePlanet'],axis=1)
test_csv=test_csv.drop(['Name','Cabin','Destination','HomePlanet'],axis=1)

In [41]:
train_csv

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,0,39.0,0,0.0,0.0,0.0,0.0,0.0,0
1,0002_01,0,24.0,0,109.0,9.0,25.0,549.0,44.0,1
2,0003_01,0,58.0,1,43.0,3576.0,0.0,6715.0,49.0,0
3,0003_02,0,33.0,0,0.0,1283.0,371.0,3329.0,193.0,0
4,0004_01,0,16.0,0,303.0,70.0,151.0,565.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,0,41.0,1,0.0,6819.0,0.0,1643.0,74.0,0
8689,9278_01,1,18.0,0,0.0,0.0,0.0,0.0,0.0,0
8690,9279_01,0,26.0,0,0.0,0.0,1872.0,1.0,0.0,1
8691,9280_01,0,32.0,0,0.0,1049.0,0.0,353.0,3235.0,0


In [5]:
Mapping={}
unique_values=train_csv['CryoSleep'].unique()
for i,value in enumerate(unique_values):
  Mapping[value]=i
train_csv['CryoSleep']=train_csv['CryoSleep'].map(Mapping)


In [6]:
Mapping={}
unique_values=train_csv['VIP'].unique()
for i,value in enumerate(unique_values):
  Mapping[value]=i
train_csv['VIP']=train_csv['VIP'].map(Mapping)

In [7]:
Mapping={}
unique_values=train_csv['Transported'].unique()
for i,value in enumerate(unique_values):
  Mapping[value]=i
train_csv['Transported']=train_csv['Transported'].map(Mapping)

In [8]:
Mapping={}
unique_values=test_csv['CryoSleep'].unique()
for i,value in enumerate(unique_values):
  Mapping[value]=i
test_csv['CryoSleep']=test_csv['CryoSleep'].map(Mapping)

In [9]:
Mapping={}
unique_values=test_csv['VIP'].unique()
for i,value in enumerate(unique_values):
  Mapping[value]=i
test_csv['VIP']=test_csv['VIP'].map(Mapping)

In [10]:
null=train_csv.isnull()
null=null.sum()
null

PassengerId       0
CryoSleep         0
Age             179
VIP               0
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

In [11]:
age_value=train_csv['Age'].mean()
train_csv.fillna(age_value,inplace=True)
Room_value=train_csv['RoomService'].mean()
train_csv.fillna(Room_value,inplace=True)
Food_value=train_csv['FoodCourt'].mean()
train_csv.fillna(Food_value,inplace=True)
Shopping_value=train_csv['ShoppingMall'].mean()
train_csv.fillna(Shopping_value,inplace=True)
Spa_value=train_csv['Spa'].mean()
train_csv.fillna(Spa_value,inplace=True)
VRDeck_value=train_csv['VRDeck'].mean()
train_csv.fillna(VRDeck_value,inplace=True)

In [12]:
model=LogisticRegression(random_state=0)
selector=sfs(model,k_features=6,forward=True,verbose=2,scoring='neg_mean_squared_error')
attributes=['CryoSleep','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']
X=train_csv[attributes].values
y=train_csv['Transported'].values
x_selector=selector.fit(X,y)



[2023-10-19 06:26:19] Features: 1/6 -- score: -0.28229403166112027
[2023-10-19 06:26:20] Features: 2/6 -- score: -0.2710215981963895
[2023-10-19 06:26:23] Features: 3/6 -- score: -0.25606696969476395
[2023-10-19 06:26:26] Features: 4/6 -- score: -0.24548286748663806
[2023-10-19 06:26:27] Features: 5/6 -- score: -0.2171835988964995
[2023-10-19 06:26:27] Features: 6/6 -- score: -0.21695364781817786

In [13]:
indices_data=x_selector.k_feature_idx_
s_attributes=[attributes[i] for i in indices_data]
s_attributes

['CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck']

In [46]:
new_attributes=['CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck']
X_new=train_csv[new_attributes].values
X_train,X_test,y_train,y_test=train_test_split(X_new,y,test_size=0.30,random_state=42)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
y_pred

array([0, 1, 1, ..., 0, 0, 1])

In [47]:
acc=accuracy_score(y_test,y_pred)
acc

0.7806748466257669

In [48]:
Mapping={
    1:'True',
    0:'False'
    }
y_pred=np.array(y_pred)
y_pred_new=[Mapping[value] for value in y_pred]


In [ ]:
y_pred_new

In [42]:
null=test_csv.isnull()
null=null.sum()
null

PassengerId       0
CryoSleep         0
Age              91
VIP               0
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
dtype: int64

In [43]:
age_value=test_csv['Age'].mean()
test_csv.fillna(age_value,inplace=True)
Room_value=test_csv['RoomService'].mean()
test_csv.fillna(Room_value,inplace=True)
Food_value=test_csv['FoodCourt'].mean()
test_csv.fillna(Food_value,inplace=True)
Shopping_value=test_csv['ShoppingMall'].mean()
test_csv.fillna(Shopping_value,inplace=True)
Spa_value=test_csv['Spa'].mean()
test_csv.fillna(Spa_value,inplace=True)
VRDeck_value=test_csv['VRDeck'].mean()
test_csv.fillna(VRDeck_value,inplace=True)

In [49]:
attributees_test=['CryoSleep', 'VIP', 'RoomService', 'FoodCourt', 'Spa', 'VRDeck']
X_for_test=test_csv[attributees_test].values
y_pred_test=model.predict(X_for_test)

In [ ]:
y_pred_test=np.array(y_pred_test)
y_pred_test=[Mapping[value] for value in y_pred_test]
y_pred_test

In [52]:
PassengerId=test_csv['PassengerId'].values
Transported=y_pred_test
file_name='Sample_Submission.csv'
with open(file_name,'w',newline='') as csvfile:
  csv_writer=csv.writer(csvfile)
  csv_writer.writerow(['PassengerId','Transported'])
  for PassengerId,Transported in zip (PassengerId,Transported):
    csv_writer.writerow(([PassengerId,Transported]))



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
